In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

### **Обязательная часть**
Вам необходимо написать функцию, которая будет основана на поиске по сайту [habr.com](https://habr.com/ru/search/). Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


### **Дополнительная часть (необязательная)**
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
word_search = ['python', 'анализ данных'] # список запросов для поиска


def return_dataframe_based_received_query_list(list_search_word, page):
    """
    Функция основана на поиске по сайту habr.com. В качестве параметра принимает
    список запросов для поиска (например, ['python', 'анализ данных']) и 
    количество страниц (например, 3) на основе материалов, попавших в результаты 
    поиска по каждому запросу, возвращать датафрейм вида:

    <дата> - <заголовок> - <ссылка на материал> - <текст материала> - 
                                                            <количество лайков>

    В рамках предполагается работа с одной или несколькими страницами результатов 
    поисковой выдачи для каждого запроса. Материалы в датафрейме не дублируются, 
    если они попадали в результаты поиска для нескольких запросов из списка. В 
    датафрейме должны быть столбцы с полным текстом найденных материалов и 
    количеством лайков
    """

    url = 'https://habr.com/ru/search/'

    params = {}

    # создаем датафрейм
    df = pd.DataFrame()

    # перебираем страницы
    for page_ in range(page+1):

        if page_ > 0:

            for i, word in enumerate(list_search_word):
            # перебор из списка значений, которые затем добавляются в параметр для 
            # запроса
                while i < len(list_search_word):
                    params['q'] = word 
                    i = i + 1

                # формируем итоговый url с нумерацией страниц, а также передачей параметра
                req = requests.get(url + 'page'+str(page_), params=params)
                time.sleep(0.3)
                soup = BeautifulSoup(req.text)

                articles = soup.find_all('article', class_='tm-articles-list__item')

                for article in articles:

                    date_ = article.find('time').text # получаем дату блога

                    # получаем название блога по условию, так как есть два варианта,
                    # один в виде строки, другое название в картинке
                    if article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'):
                        title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text

                    elif article.find('h2', class_='tm-megapost-snippet__title'):
                        title = article.find('h2', class_='tm-megapost-snippet__title').text

                    # создаем список, в которые будут добавляться ссылки на 
                    # блоги/новости для дальнейшего получения по ним текста из блога
                    all_links = []

                    # получаем ссылку на блог
                    if article.find('a', class_='tm-article-snippet__title-link'):
                        link = 'https://habr.com' + article.find('a', class_='tm-article-snippet__title-link').get('href') 
                        all_links.append(link)

                    elif article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card'):
                        link = 'https://habr.com' + article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card').get('href')
                        all_links.append(link)

                    # получаем лайки
                    number_likes = article.find('div', class_='tm-votes-meter tm-data-icons__item').text

                    # формируем словарь для дальнейшего добавления DataFrame
                    row = {'date': date_, 'title': title, 'link': link, 'number_likes': number_likes} 
                    
                    # получаем ссылки на каждый блог
                    for links in all_links:
                        soup = BeautifulSoup(requests.get(links).text)
                        time.sleep(0.33)
                        
                        # получаем текст блога по каждой ссылке
                        if soup.find('div', 't-records'):
                            text = soup.find('div', 't-records').text.strip()

                        elif soup.find('div', 'article-formatted-body article-formatted-body article-formatted-body_version-1'):
                            text = soup.find('div', 'article-formatted-body article-formatted-body article-formatted-body_version-1').text.strip()

                        elif soup.find('div', 'article-formatted-body article-formatted-body article-formatted-body_version-2'):
                            text = soup.find('div', 'article-formatted-body article-formatted-body article-formatted-body_version-2').text.strip()
                        
                        row.update({'text': text}) # текст блога добавляем в словарь

                    # добавляем полученные данные из словаря row в созданный датафрейм
                    df = pd.concat([df, pd.DataFrame([row])])
    
    # убираем дубликаты, если такие есть по названию блога и текста, и выводим 
    # итоговый результат согласно условию функции
    return df.drop_duplicates(subset=['title', 'text']).reset_index(drop=True)


return_dataframe_based_received_query_list(word_search, 4)


,date,title,link,number_likes,text
0,вчера в 16:06,Инструменты для GraphQL клиента на Python,https://habr.com/ru/post/707374/,Всего голосов 7: ↑5 и ↓2 +3,Речь в статье пойдет о специальных инструмента...
1,13 декабря 2021 в 09:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,Всего голосов 11: ↑10 и ↓1 +9,21 декабря Слёрм проведёт открытый урок «ChatO...
2,20 января в 18:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,Всего голосов 14: ↑12 и ↓2 +10,"Курс нацелен дать максимальную пользу, поэтому..."
3,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,Всего голосов 19: ↑19 и ↓0 +19,"20 апреля 2020 года, спустя почти десять лет п..."
4,6 июля 2021 в 13:29,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,Всего голосов 3: ↑3 и ↓0 +3,"Первый открытый онлайн-митап сообщества, для к..."
...,...,...,...,...,...
154,24 июня 2016 в 23:12,"Достучаться до госорганов или что делать, если...",https://habr.com/ru/company/infoculture/blog/3...,Всего голосов 20: ↑17 и ↓3 +14,источник картинки: southriverrestoration.com/w...
155,24 апреля 2017 в 13:46,Приглашаем на IV конференцию по практическому ...,https://habr.com/ru/company/flyelephant/blog/3...,Всего голосов 30: ↑26 и ↓4 +22,Команда FlyElephant приглашает всех 13 мая в О...
156,10 июля 2017 в 16:14,Лекция о двух библиотеках Яндекса для работы с...,https://habr.com/ru/company/yandex/blog/332688/,Всего голосов 35: ↑34 и ↓1 +33,Пару недель назад в Яндексе прошла встреча PyD...
157,22 октября 2018 в 13:29,Бизнес на персональных данных: как добиться ус...,https://habr.com/ru/company/digitalrightscente...,Всего голосов 9: ↑8 и ↓1 +7,«Данные – нефть цифровой экономики» — выражени...
